In [1]:
import polars as pl
import glob
from datetime import datetime

In [2]:
path_to_items = "data/sales_pers.item_chunk_0.parquet"
path_to_purchases = "data/sales_pers.purchase_history_daily_chunk_*.parquet"
path_to_users = "data/sales_pers.user_chunk_*.parquet"

item_df = pl.scan_parquet(path_to_items)
user_df = pl.scan_parquet(glob.glob(path_to_users))
purchase_df = pl.scan_parquet(glob.glob(path_to_purchases))

# A. Hãy thống kê những sản phẩm hay mua chung và số lần mua chung: item 1 | item 2 | #cooc

In [ ]:
(
    purchase_df
    .join(
        purchase_df, on=['user_id', 'timestamp']
    )
    .filter(
        pl.col('item_id') < pl.col('item_id_right')
    )
    .select([
        pl.col('item_id').alias('item_1'),
        pl.col('item_id_right').alias('item_2')
    ])
    .sink_parquet('intermediate_pairs_optimized.parquet', compression='zstd')
)

print("Đã ghi xong file trung gian")

Đã ghi xong file trung gian


In [ ]:
final_co_occurrence = (
    pl.scan_parquet('intermediate_pairs_optimized.parquet')
    .group_by(['item_1', 'item_2'])
    .agg(pl.len().alias('#cooc'))
    .sort('#cooc', descending=True)
    .collect()
)

print("\nBảng tần suất mua chung cuối cùng:")
print(final_co_occurrence)
del final_co_occurrence


Bảng tần suất mua chung cuối cùng:
shape: (7_417_085, 3)
┌───────────────┬───────────────┬───────┐
│ item_1        ┆ item_2        ┆ #cooc │
│ ---           ┆ ---           ┆ ---   │
│ str           ┆ str           ┆ u32   │
╞═══════════════╪═══════════════╪═══════╡
│ 2803000000011 ┆ 2803000000013 ┆ 78241 │
│ 2803000000012 ┆ 2803000000013 ┆ 53481 │
│ 2803000000011 ┆ 2803000000012 ┆ 52849 │
│ 2803000000010 ┆ 2803000000012 ┆ 34205 │
│ 2803000000010 ┆ 2803000000013 ┆ 32656 │
│ …             ┆ …             ┆ …     │
│ 4553000000002 ┆ 5073000000002 ┆ 1     │
│ 0919000000118 ┆ 2166000000001 ┆ 1     │
│ 1753000000015 ┆ 5537000000012 ┆ 1     │
│ 2588000000002 ┆ 3793000000010 ┆ 1     │
│ 1396000000039 ┆ 3514000000014 ┆ 1     │
└───────────────┴───────────────┴───────┘


# B. Hãy dự đoán tuổi của em bé dựa trên:

- Thông tin "age_group" của bảng item: Ngày đầu tiên mua

- Thông tin sữa có chữ "Step 1": Ngày đầu tiên mua

- Thong tin sữa có chữ "Mom": Ngày cuối cùng mua

Tạo bảng dự đoán: | customer_id | first_date_buy_step 1 | age_by_step1 | first_date_buy_age_group_0-3M | age_by_age_group | last_date_buy_milk4mom | age_by_milk4mom.

## Sữa step 1

In [3]:
pl.Config.set_fmt_str_lengths(100000)

regex_pattern = r"step\s*1"
item_step1_df = item_df.filter(
    pl.col('category').str.to_lowercase().str.contains(regex_pattern),
    pl.col('category_l1').str.to_lowercase().str.contains('sữa')
).select(
    pl.col('item_id'),
)


## Sữa mom

In [4]:
regex_pattern = r"\bmom\b"
item_mom_df = item_df.filter(
    pl.col('category').str.to_lowercase().str.contains(regex_pattern),
    pl.col('category_l1').str.to_lowercase().str.contains('sữa')
).select(
    pl.col('item_id'),
)

## Hàm chuyển đổi age group

In [ ]:
# --- Định nghĩa Hằng số ---
MONTHS_IN_YEAR = 12
MOM_AGE_MONTHS = 0

# --- Hàm mã hóa age_group về số tháng nhỏ nhất (đã sửa) ---
def age_group_to_min_months_lazy(col: pl.Expr) -> pl.Expr:
    # 1. Chuẩn hóa về chuỗi, loại bỏ khoảng trắng và 'Từ '
    cleaned_group = col.str.replace_all(r"[\s\u00A0]+", "").str.replace(r"^Từ", "")

    # 2. Xử lý trường hợp "Mẹ" -> 0 tháng
    is_mom = cleaned_group.eq_missing("Mẹ")

    # 3. Trích xuất số và đơn vị (M/Y) một cách linh hoạt hơn
    # Lấy số ở ĐẦU chuỗi (ví dụ: lấy '0' từ '0-12M')
    age_match = cleaned_group.str.extract(r"^(\d+)", 1)
    # Lấy đơn vị ở CUỐI chuỗi (ví dụ: lấy 'M' từ '0-12M')
    unit_match = cleaned_group.str.extract(r"([MY])$", 1)

    # Cast phần số sang Int32
    age_value_int = age_match.cast(pl.Int32, strict=False)

    # Chuyển đổi sang số tháng (M)
    min_months = (
        pl.when(unit_match.eq_missing("Y"))
        .then(age_value_int * MONTHS_IN_YEAR)
        .when(unit_match.eq_missing("M"))
        .then(age_value_int)
        .otherwise(pl.lit(None).cast(pl.Int32))
    )
    
    # Kết hợp lại với trường hợp "Mẹ"
    final_min_months = (
        pl.when(is_mom)
        .then(pl.lit(MOM_AGE_MONTHS))
        .otherwise(min_months)
    )

    return final_min_months.alias("age_group_month")

## Age group

In [6]:
item_age_group_df = item_df.filter(
    pl.col('age_group') != "Không xác định"
).select(
    pl.col('item_id'),
    age_group_to_min_months_lazy(pl.col("age_group"))
)

## Thống kê

In [7]:
# Ngày đầu mua age group
age_group_purchases = purchase_df.join(
    item_age_group_df,
    on='item_id', 
    how='inner'
)

first_buy_age_group = age_group_purchases.sort(['created_date', 'age_group_month'], nulls_last=True).group_by('user_id').first().select(
    pl.col('user_id'),
    pl.col('created_date').alias('first_date_buy_age_group'),
    pl.col('age_group_month')
)


# Ngày đầu mua sữa step1
step1_purchases = purchase_df.join(
    item_step1_df.select('item_id'), 
    on='item_id', 
    how='inner'
)
first_buy_step1 = step1_purchases.group_by('user_id').agg(
    pl.col('created_date').min().alias('first_date_buy_step1')
)

# Ngày cuối mua sữa mom
mom_purchases = purchase_df.join(
    item_mom_df.select('item_id'), 
    on='item_id', 
    how='inner'
)
last_buy_mom = mom_purchases.group_by('user_id').agg(
    pl.col('created_date').max().alias('last_date_buy_milk4mom')
)

# Join các bảng lại với nhau
df = first_buy_step1.join(
    last_buy_mom, 
    on='user_id', 
    how='full'
).with_columns(
    pl.coalesce([pl.col('user_id'), pl.col('user_id_right')]).alias('user_id_merged')
).select(
    pl.col('user_id_merged').alias('user_id'),
    pl.col('first_date_buy_step1'),
    pl.col('last_date_buy_milk4mom')
)

df = df.join(
    first_buy_age_group, 
    on='user_id', 
    how='full'
).with_columns(
    pl.coalesce([pl.col('user_id'), pl.col('user_id_right')]).alias('user_id_merged')
).select(
    pl.col('user_id_merged').alias('user_id'),
    pl.col('first_date_buy_step1'),
    pl.col('last_date_buy_milk4mom'),
    pl.col('first_date_buy_age_group'),
    pl.col('age_group_month')
)

# Dự đoán tuổi
today = datetime.now()
final_df = df.with_columns(
    (pl.when(pl.col('first_date_buy_step1').is_not_null())
       .then((pl.lit(today) - pl.col('first_date_buy_step1')).dt.total_days() / 30)
       .otherwise(None)
       .alias('age_by_step1')),
    
    (pl.when(pl.col('last_date_buy_milk4mom').is_not_null())
       .then((pl.lit(today) - pl.col('last_date_buy_milk4mom')).dt.total_days() / 30)
       .otherwise(None)
       .alias('age_by_milk4mom')),
       
    (pl.when(pl.col('first_date_buy_age_group').is_not_null())
       .then(((pl.lit(today) - pl.col('first_date_buy_age_group')).dt.total_days() / 30)  + pl.col('age_group_month'))
       .otherwise(None)
       .alias('age_by_age_group'))
)

final_df = final_df.join(
    user_df.select(['user_id', 'customer_id']),
    on='user_id',
    how="right"
).select(
    pl.col(['customer_id', 'first_date_buy_step1', 'age_by_step1', 'first_date_buy_age_group', 'age_by_age_group', 'last_date_buy_milk4mom', 'age_by_milk4mom'])
).filter(
    pl.any_horizontal(
        pl.col(['age_by_age_group', 'age_by_step1', 'age_by_milk4mom']).is_not_null()
    )
).collect()

print(final_df)

shape: (1_829_593, 7)
┌─────────────┬──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ customer_id ┆ first_date_b ┆ age_by_step ┆ first_date_ ┆ age_by_age_ ┆ last_date_b ┆ age_by_milk │
│ ---         ┆ uy_step1     ┆ 1           ┆ buy_age_gro ┆ group       ┆ uy_milk4mom ┆ 4mom        │
│ i32         ┆ ---          ┆ ---         ┆ up          ┆ ---         ┆ ---         ┆ ---         │
│             ┆ datetime[μs] ┆ f64         ┆ ---         ┆ f64         ┆ datetime[μs ┆ f64         │
│             ┆              ┆             ┆ datetime[μs ┆             ┆ ]           ┆             │
│             ┆              ┆             ┆ ]           ┆             ┆             ┆             │
╞═════════════╪══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 14732       ┆ null         ┆ null        ┆ 2024-11-29  ┆ 19.666667   ┆ null        ┆ null        │
│             ┆              ┆             ┆ 20:22:55.48 ┆           